In [48]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

df = pd.read_csv('total_with_chute_time.csv')

In [49]:
df.columns

Index(['tdate', 'putime', 'disptime', 'enrtime', 'atstime', 'tratime',
       'atdtime', 'yearwithrc', 'Call Type', 'veh', 'Sending', 'PU City',
       'atddate', 'Mileage - Loaded', 'Receiving', 'D State', 'Diagnosis',
       'Cancel Reason', 'PU City.1', 'PU State', 'Crew1', 'Role1', 'Crew2',
       'Role2', 'Crew3', 'Role3', 'Tail Number', 'Status', 'Crew4', 'Role4',
       'disptime_dt', 'enrtime_dt', 'adjusted_enrtime_dt', 'chute_time',
       'chute_time_minutes'],
      dtype='object')

In [50]:
pd.set_option('display.max_columns', None)

cols = ['tdate', 'putime', 'disptime', 'enrtime', 'atstime', 'tratime',
       'atdtime', 'yearwithrc', 'Call Type', 'veh', 'Sending', 'PU City',
       'atddate', 'Mileage - Loaded', 'Receiving', 'D State', 'Diagnosis',
       'Cancel Reason', 'PU City.1', 'PU State', 'Crew1', 'Role1', 'Crew2',
       'Role2', 'Crew3', 'Role3', 'Tail Number',
       'chute_time_minutes']

# df = df.drop(columns=cols)


df.head()

,tdate,putime,disptime,enrtime,atstime,tratime,atdtime,yearwithrc,Call Type,veh,Sending,PU City,atddate,Mileage - Loaded,Receiving,D State,Diagnosis,Cancel Reason,PU City.1,PU State,Crew1,Role1,Crew2,Role2,Crew3,Role3,Tail Number,Status,Crew4,Role4,disptime_dt,enrtime_dt,adjusted_enrtime_dt,chute_time,chute_time_minutes
0,2023-01-01,15:39:41,15:41:36,15:59:03,16:09:19,16:29:54,17:15:04,23-00060,LOM INTERFACILITY,Lewiston,Mid Coast Hospital,BRUNSWICK,2023-01-01,120.0,Massachusetts General Hospital,MA,Cancer Leukemia,<NONE>,Cumberland,Maine,"Br...,Se...",LOM Medic,"Si...,Pa...",LOM Medic,"Ha...,Ka...",LOM RW Pilot,Unknown,Unknown,Unknown,Unknown,1900-01-01 15:41:36,1900-01-01 15:59:03,1900-01-01 15:59:03,0 days 00:17:27,17.450000
1,2023-01-01,12:47:17,12:48:33,13:00:26,13:17:22,14:07:58,14:31:20,23-00043,LOM INTERFACILITY,Lewiston,Franklin Memorial Hospital,FARMINGTON,2023-01-01,68.0,Maine Medical Center,ME,Other arterial embolism and thrombosis of abd ...,<NONE>,Franklin,Maine,"Br...,Se...",LOM Medic,"Si...,Pa...",LOM Medic,"Ha...,Ka...",LOM RW Pilot,Unknown,Unknown,Unknown,Unknown,1900-01-01 12:48:33,1900-01-01 13:00:26,1900-01-01 13:00:26,0 days 00:11:53,11.883333
2,2023-01-01,17:44:06,17:44:32,18:08:27,18:19:35,20:34:14,20:47:08,23-00074,LOM INTERFACILITY,Bangor,Maine Coast Memorial Hospital,ELLSWORTH,2023-01-01,25.0,Eastern Maine Medical Center,ME,"Respiratroy failure, unspecified, unspecified ...",<NONE>,Hancock,Maine,"Me...,Br...",LOM Medic,"Ma...,Ve...",LOM RN,"Sm...,Du...",LOM RW Pilot,Unknown,Unknown,Unknown,Unknown,1900-01-01 17:44:32,1900-01-01 18:08:27,1900-01-01 18:08:27,0 days 00:23:55,23.916667
3,2023-01-02,16:49:03,16:49:12,17:01:43,17:14:40,17:31:30,17:45:34,23-00190,LOM SCENE,Lewiston,Bridgton Hospital,BRIDGTON,2023-01-02,36.0,Maine Medical Center,ME,"Unspecified injury of abdomen, initial enc",<NONE>,Cumberland,Maine,"Si...,Pa...",LOM Medic,"St...,Ba...",LOM RN,"Ha...,Ka...",LOM RW Pilot,Unknown,Unknown,Unknown,Unknown,1900-01-01 16:49:12,1900-01-01 17:01:43,1900-01-01 17:01:43,0 days 00:12:31,12.516667
4,2023-01-02,18:27:02,19:01:42,19:18:14,19:30:27,20:37:10,21:07:21,23-00197,LOM INTERFACILITY,Bangor,Waldo County General Hospital,BELFAST,2023-01-02,83.0,Maine Medical Center,ME,Myxedema coma,<NONE>,Waldo,Maine,"Ba...,Cr...",LOM Medic,"Sh...,Lo...",LOM RN,"Bu...,Sh...",LOM RW Pilot,Unknown,Unknown,Unknown,Unknown,1900-01-01 19:01:42,1900-01-01 19:18:14,1900-01-01 19:18:14,0 days 00:16:32,16.533333


In [51]:
df = df[(df['chute_time_minutes'] >= 5) & (df['chute_time_minutes'] <= 30)]
df.shape

(14142, 35)

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14142 entries, 0 to 17379
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   tdate                14142 non-null  object 
 1   putime               14142 non-null  object 
 2   disptime             14142 non-null  object 
 3   enrtime              14142 non-null  object 
 4   atstime              14142 non-null  object 
 5   tratime              14142 non-null  object 
 6   atdtime              14142 non-null  object 
 7   yearwithrc           14142 non-null  object 
 8   Call Type            14142 non-null  object 
 9   veh                  14142 non-null  object 
 10  Sending              14142 non-null  object 
 11  PU City              14142 non-null  object 
 12  atddate              14142 non-null  object 
 13  Mileage - Loaded     14142 non-null  float64
 14  Receiving            14142 non-null  object 
 15  D State              14142 non-null 

In [53]:
cols = ['Call Type', 'veh', 'Sending', 'PU City', 'Mileage - Loaded', 'Receiving', 'D State', 'Diagnosis',
       'Cancel Reason', 'PU City.1', 'PU State', 'Crew1', 'Role1', 'Crew2',
       'Role2', 'Crew3', 'Role3', 'Tail Number'
       ]

# Assuming your DataFrame is named df
for col in cols:
    # Print the column name
    print(f"Column: {col}")
    # Print unique values in the column
    print(df[col].unique())
    # Optionally, add a blank line for better readability
    print()


Column: Call Type
['LOM INTERFACILITY' 'LOM SCENE' 'LOM CCT GROUND' 'NL CCT']

Column: veh
['Lewiston' 'Bangor' 'Sanford']

Column: Sending
['Mid Coast Hospital' 'Franklin Memorial Hospital'
 'Maine Coast Memorial Hospital' 'Bridgton Hospital'
 'Waldo County General Hospital' 'AR Gould Hospital'
 'Rumford Community Hospital' 'Down East Community Hospital'
 'Stephens Memorial' 'Maine General - Harold Alfond Center for Care'
 'Lincoln Health Care - Miles Memorial Hospital' '<None>'
 'Memorial Hospital' 'Blue Hill Memorial Hospital'
 'Millinocket Regional Hospital' 'Houlton Regional Hospital'
 'Redington-Fairview General Hospital' 'Mt Desert Island Hospital'
 'Vinalhaven Clinic' 'Inland Hospital' 'Penobscot Bay Medical Center'
 'Carrabassett - Sugarloaf Regional' 'Calais Regional Hospital'
 'Cary Medical Center' 'Central Maine Medical Center'
 'Penobscot Valley Hospital' 'Sebasticook Valley Hospital'
 'Auburn-Lewiston Airport (KLEW)' 'Mary Talbot Field (ME55)'
 'SMMC Southern Maine Health

In [54]:
duplicate_rows = df.duplicated(subset=['tdate', 'putime', 'disptime', 'enrtime', 'atstime', 'tratime', 'atdtime'], keep=False)
duplicate_count = duplicate_rows.sum()
duplicate_count

2296

In [55]:
df_cleaned = df.drop_duplicates(subset=['tdate', 'putime', 'disptime', 'enrtime', 'atstime', 'tratime', 'atdtime'])

duplicate_count, df_cleaned.shape

(2296, (12994, 35))

In [56]:
df.shape

(14142, 35)

In [57]:


# Assuming your DataFrame is named df
for col in df.columns:
    # Print the column name
    print(f"Column: {col}")
    # Print unique values in the column
    print(df[col].unique())
    # Optionally, add a blank line for better readability
    print()


Column: tdate
['2023-01-01' '2023-01-02' '2023-01-04' ... '2023-12-06' '2023-12-13'
 '2023-12-02']

Column: putime
['15:39:41' '12:47:17' '17:44:06' ... '21:24:19' '22:51:58' '10:37:03']

Column: disptime
['15:41:36' '12:48:33' '17:44:32' ... '11:17:15' '22:52:21' '10:37:49']

Column: enrtime
['15:59:03' '13:00:26' '18:08:27' ... '11:31:02' '23:11:55' '10:55:05']

Column: atstime
['16:09:19' '13:17:22' '18:19:35' ... '23:58:04' '22:10:32' '23:31:09']

Column: tratime
['16:29:54' '14:07:58' '20:34:14' ... '12:49:16' '23:54:28' '11:45:23']

Column: atdtime
['17:15:04' '14:31:20' '20:47:08' ... '23:18:52' '13:18:42' '00:47:11']

Column: yearwithrc
['23-00060' '23-00043' '23-00074' ... '23-56164' '23-56232' '23-53107']

Column: Call Type
['LOM INTERFACILITY' 'LOM SCENE' 'LOM CCT GROUND' 'NL CCT']

Column: veh
['Lewiston' 'Bangor' 'Sanford']

Column: Sending
['Mid Coast Hospital' 'Franklin Memorial Hospital'
 'Maine Coast Memorial Hospital' 'Bridgton Hospital'
 'Waldo County General Hospita

In [58]:
df.columns

Index(['tdate', 'putime', 'disptime', 'enrtime', 'atstime', 'tratime',
       'atdtime', 'yearwithrc', 'Call Type', 'veh', 'Sending', 'PU City',
       'atddate', 'Mileage - Loaded', 'Receiving', 'D State', 'Diagnosis',
       'Cancel Reason', 'PU City.1', 'PU State', 'Crew1', 'Role1', 'Crew2',
       'Role2', 'Crew3', 'Role3', 'Tail Number', 'Status', 'Crew4', 'Role4',
       'disptime_dt', 'enrtime_dt', 'adjusted_enrtime_dt', 'chute_time',
       'chute_time_minutes'],
      dtype='object')

In [59]:
df.shape

(14142, 35)

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14142 entries, 0 to 17379
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   tdate                14142 non-null  object 
 1   putime               14142 non-null  object 
 2   disptime             14142 non-null  object 
 3   enrtime              14142 non-null  object 
 4   atstime              14142 non-null  object 
 5   tratime              14142 non-null  object 
 6   atdtime              14142 non-null  object 
 7   yearwithrc           14142 non-null  object 
 8   Call Type            14142 non-null  object 
 9   veh                  14142 non-null  object 
 10  Sending              14142 non-null  object 
 11  PU City              14142 non-null  object 
 12  atddate              14142 non-null  object 
 13  Mileage - Loaded     14142 non-null  float64
 14  Receiving            14142 non-null  object 
 15  D State              14142 non-null 

In [61]:
# Columns to drop
columns_to_drop = ['yearwithrc', 'Cancel Reason', 'adjusted_enrtime_dt', 'disptime_dt', 'enrtime_dt','chute_time']

# Dropping the columns
df_dropped = df.drop(columns=columns_to_drop)

# Check the remaining columns
print(df_dropped.columns)


Index(['tdate', 'putime', 'disptime', 'enrtime', 'atstime', 'tratime',
       'atdtime', 'Call Type', 'veh', 'Sending', 'PU City', 'atddate',
       'Mileage - Loaded', 'Receiving', 'D State', 'Diagnosis', 'PU City.1',
       'PU State', 'Crew1', 'Role1', 'Crew2', 'Role2', 'Crew3', 'Role3',
       'Tail Number', 'Status', 'Crew4', 'Role4', 'chute_time_minutes'],
      dtype='object')


In [62]:
# Convert 'tdate' column to datetime type
df_dropped['tdate'] = pd.to_datetime(df_dropped['tdate'])

# Function to convert time string to minutes past midnight
def time_to_minutes(time_str):
    time_parts = time_str.split(':')
    return int(time_parts[0]) * 60 + int(time_parts[1])

# Time columns to process
time_columns = ['putime', 'disptime', 'enrtime', 'atstime', 'tratime', 'atdtime']

# Apply the conversion to all time columns
for col in time_columns:
    df_dropped[col] = df_dropped[col].apply(time_to_minutes)

df_dropped.head()

,tdate,putime,disptime,enrtime,atstime,tratime,atdtime,Call Type,veh,Sending,PU City,atddate,Mileage - Loaded,Receiving,D State,Diagnosis,PU City.1,PU State,Crew1,Role1,Crew2,Role2,Crew3,Role3,Tail Number,Status,Crew4,Role4,chute_time_minutes
0,2023-01-01,939,941,959,969,989,1035,LOM INTERFACILITY,Lewiston,Mid Coast Hospital,BRUNSWICK,2023-01-01,120.0,Massachusetts General Hospital,MA,Cancer Leukemia,Cumberland,Maine,"Br...,Se...",LOM Medic,"Si...,Pa...",LOM Medic,"Ha...,Ka...",LOM RW Pilot,Unknown,Unknown,Unknown,Unknown,17.450000
1,2023-01-01,767,768,780,797,847,871,LOM INTERFACILITY,Lewiston,Franklin Memorial Hospital,FARMINGTON,2023-01-01,68.0,Maine Medical Center,ME,Other arterial embolism and thrombosis of abd ...,Franklin,Maine,"Br...,Se...",LOM Medic,"Si...,Pa...",LOM Medic,"Ha...,Ka...",LOM RW Pilot,Unknown,Unknown,Unknown,Unknown,11.883333
2,2023-01-01,1064,1064,1088,1099,1234,1247,LOM INTERFACILITY,Bangor,Maine Coast Memorial Hospital,ELLSWORTH,2023-01-01,25.0,Eastern Maine Medical Center,ME,"Respiratroy failure, unspecified, unspecified ...",Hancock,Maine,"Me...,Br...",LOM Medic,"Ma...,Ve...",LOM RN,"Sm...,Du...",LOM RW Pilot,Unknown,Unknown,Unknown,Unknown,23.916667
3,2023-01-02,1009,1009,1021,1034,1051,1065,LOM SCENE,Lewiston,Bridgton Hospital,BRIDGTON,2023-01-02,36.0,Maine Medical Center,ME,"Unspecified injury of abdomen, initial enc",Cumberland,Maine,"Si...,Pa...",LOM Medic,"St...,Ba...",LOM RN,"Ha...,Ka...",LOM RW Pilot,Unknown,Unknown,Unknown,Unknown,12.516667
4,2023-01-02,1107,1141,1158,1170,1237,1267,LOM INTERFACILITY,Bangor,Waldo County General Hospital,BELFAST,2023-01-02,83.0,Maine Medical Center,ME,Myxedema coma,Waldo,Maine,"Ba...,Cr...",LOM Medic,"Sh...,Lo...",LOM RN,"Bu...,Sh...",LOM RW Pilot,Unknown,Unknown,Unknown,Unknown,16.533333


In [63]:
categorical_cols = ['Call Type', 'veh', 'D State', 'PU State', 'Status', 'Sending', 'Receiving', 'PU City', 'PU City.1', 'Tail Number']  # Add more as needed


In [64]:
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)


In [65]:
df_encoded.columns

Index(['tdate', 'putime', 'disptime', 'enrtime', 'atstime', 'tratime',
       'atdtime', 'yearwithrc', 'atddate', 'Mileage - Loaded',
       ...
       'Tail Number_N285BP', 'Tail Number_N517MT', 'Tail Number_N692SH',
       'Tail Number_N710SB', 'Tail Number_N901CM', 'Tail Number_N901EM',
       'Tail Number_N901LF', 'Tail Number_N901WM', 'Tail Number_N901XM',
       'Tail Number_Unknown'],
      dtype='object', length=634)

In [66]:
# List all columns in the DataFrame
columns = df_encoded.columns

# Filter columns that end with 'Unknown'
unknown_columns = [col for col in columns if col.endswith('Unknown')]

# Drop these columns from the DataFrame
df_encoded.drop(columns=unknown_columns, inplace=True)


In [67]:
df_encoded.columns

Index(['tdate', 'putime', 'disptime', 'enrtime', 'atstime', 'tratime',
       'atdtime', 'yearwithrc', 'atddate', 'Mileage - Loaded',
       ...
       'Tail Number_LF4', 'Tail Number_N285BP', 'Tail Number_N517MT',
       'Tail Number_N692SH', 'Tail Number_N710SB', 'Tail Number_N901CM',
       'Tail Number_N901EM', 'Tail Number_N901LF', 'Tail Number_N901WM',
       'Tail Number_N901XM'],
      dtype='object', length=630)

In [68]:
df_encoded['Diagnosis'].unique()

array(['Cancer Leukemia',
       'Other arterial embolism and thrombosis of abd aort',
       'Respiratroy failure, unspecified, unspecified whet', ...,
       'Other specified injury of popliteal artery, right',
       'Staphylococcal scalded skin syndrome',
       'Complete traumatic amputation of right lower leg,'], dtype=object)

In [69]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
import pandas as pd

# Let's assume df_encoded is your DataFrame and it's ready for preprocessing
# Identify categorical and numerical columns (excluding 'Diagnosis' for now)
categorical_cols = [col for col in df_encoded.columns if df_encoded[col].dtype == 'object' and col != 'Diagnosis']
numerical_cols = [col for col in df_encoded.columns if df_encoded[col].dtype in ['int64', 'float64']]

# Define preprocessing for numerical columns (here we just scale them)
numerical_transformer = StandardScaler()

# Define preprocessing for categorical columns (encode them)
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create the preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Apply the preprocessing, excluding the 'Diagnosis' column for now
X_preprocessed = preprocessor.fit_transform(df_encoded.drop(['Diagnosis'], axis=1))


In [71]:
from scipy.sparse import hstack

# Combine the preprocessed features with the TF-IDF vectorized 'Diagnosis' column
X_final = hstack([X_preprocessed, tfidf_features])


In [72]:
X_final.shape

(14142, 128994)

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import numpy as np

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_final, df_encoded['chute_time_minutes'], test_size=0.2, random_state=42)

# Initialize and train the Ridge regression model
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)

# Predicting on the test set
y_pred = ridge_model.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"RMSE: {rmse}")


RMSE: 0.007239443106902682


In [77]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge

# Initialize the Ridge regression model
ridge_model_cv = Ridge(alpha=1.0)

# Perform cross-validation
cv_scores_ridge = cross_val_score(ridge_model_cv, X_final, df_encoded['chute_time_minutes'], cv=5, scoring='neg_mean_squared_error')

# Calculate RMSE from cross-validation scores
rmse_scores_ridge = np.sqrt(-cv_scores_ridge)
print(f"Ridge Regression RMSE (cross-validation): {rmse_scores_ridge.mean()} ± {rmse_scores_ridge.std()}")


Ridge Regression RMSE (cross-validation): 0.007452907204030254 ± 0.001114627557854674


In [78]:
from sklearn.ensemble import GradientBoostingRegressor

# Initialize the Gradient Boosting regressor model
gb_model_cv = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Perform cross-validation
cv_scores_gb = cross_val_score(gb_model_cv, X_final, df_encoded['chute_time_minutes'], cv=5, scoring='neg_mean_squared_error')

# Calculate RMSE from cross-validation scores
rmse_scores_gb = np.sqrt(-cv_scores_gb)
print(f"Gradient Boosting Regressor RMSE (cross-validation): {rmse_scores_gb.mean()} ± {rmse_scores_gb.std()}")


Gradient Boosting Regressor RMSE (cross-validation): 0.047181790621812346 ± 0.0014571433859364117


In [74]:
from sklearn.decomposition import TruncatedSVD

# Initialize and apply Truncated SVD
svd = TruncatedSVD(n_components=300)  # Adjust the number of components
X_reduced = svd.fit_transform(X_final)

# Now, use X_reduced for train-test split and model training


In [76]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import numpy as np

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reduced, df_encoded['chute_time_minutes'], test_size=0.2, random_state=42)

# Initialize and train the Ridge regression model
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)

# Predicting on the test set
y_pred = ridge_model.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"RMSE: {rmse}")


RMSE: 0.0034983690241823335


In [79]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge

# Initialize the Ridge regression model
ridge_model_cv = Ridge(alpha=1.0)

# Perform cross-validation
cv_scores_ridge = cross_val_score(ridge_model_cv, X_reduced, df_encoded['chute_time_minutes'], cv=5, scoring='neg_mean_squared_error')

# Calculate RMSE from cross-validation scores
rmse_scores_ridge = np.sqrt(-cv_scores_ridge)
print(f"Ridge Regression RMSE (cross-validation): {rmse_scores_ridge.mean()} ± {rmse_scores_ridge.std()}")


Ridge Regression RMSE (cross-validation): 0.004114906575199648 ± 0.0007881144445012089


In [80]:
from sklearn.ensemble import GradientBoostingRegressor

# Initialize the Gradient Boosting regressor model
gb_model_cv = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Perform cross-validation
cv_scores_gb = cross_val_score(gb_model_cv, X_reduced, df_encoded['chute_time_minutes'], cv=5, scoring='neg_mean_squared_error')

# Calculate RMSE from cross-validation scores
rmse_scores_gb = np.sqrt(-cv_scores_gb)
print(f"Gradient Boosting Regressor RMSE (cross-validation): {rmse_scores_gb.mean()} ± {rmse_scores_gb.std()}")


Gradient Boosting Regressor RMSE (cross-validation): 0.4809424794862923 ± 0.04923426593001523
